# Open AI မော်ဒယ်များကို အတိကျစွာ ပြင်ဆင်ခြင်း

ဤ notebook သည် Open AI မှ ပေးအပ်ထားသော [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) လမ်းညွှန်ချက်များအပေါ် အခြေခံထားသည်။

Fine-tuning သည် သင့်လျော်သော အသုံးပြုမှု သို့မဟုတ် အခြေအနေအတွက် သက်ဆိုင်သော အချက်အလက်နှင့် အကြောင်းအရာများဖြင့် ထပ်မံလေ့ကျင့်ခြင်းဖြင့် မူလမော်ဒယ်များ၏ စွမ်းဆောင်ရည်ကို တိုးတက်စေသည်။ _few shot learning_ နှင့် _retrieval augmented generation_ ကဲ့သို့သော prompt engineering နည်းလမ်းများက သင့်ရဲ့ default prompt ကို သက်ဆိုင်ရာ အချက်အလက်များဖြင့် တိုးမြှင့်နိုင်ပေမယ့်၊ ဤနည်းလမ်းများသည် ရည်ရွယ်ထားသော မူလမော်ဒယ်၏ max token window အရွယ်အစားဖြင့် ကန့်သတ်ထားသည်။

Fine-tuning ဖြင့် မော်ဒယ်ကို လိုအပ်သော ဒေတာဖြင့် ထပ်မံလေ့ကျင့်ခြင်းဖြစ်ပြီး (max token window ထဲသို့ ထည့်နိုင်သည့် ဥပမာများထက် ပိုမိုများစွာ အသုံးပြုနိုင်စေသည်) inference အချိန်တွင် ဥပမာများ မလိုအပ်တော့သော _custom_ မော်ဒယ်ဗားရှင်းကို ထုတ်လုပ်သည်။ ၎င်းသည် prompt ဒီဇိုင်း၏ ထိရောက်မှုကို တိုးတက်စေပြီး (token window ကို အခြားအရာများအတွက် ပိုမိုလွတ်လပ်စွာ အသုံးပြုနိုင်ခြင်း) inference အချိန်တွင် မော်ဒယ်သို့ ပို့ရမည့် token အရေအတွက်ကို လျော့ချခြင်းဖြင့် ကုန်ကျစရိတ်ကိုလည်း တိုးတက်စေနိုင်သည်။

Fine tuning တွင် အဆင့် ၄ ဆင့်ရှိသည်။
1. လေ့ကျင့်ရေး ဒေတာကို ပြင်ဆင်ပြီး တင်သွင်းပါ။
1. လေ့ကျင့်ရေး အလုပ်ကို ပြုလုပ်၍ fine-tuned မော်ဒယ်ရယူပါ။
1. fine-tuned မော်ဒယ်ကို သုံးသပ်ပြီး အရည်အသွေးအတွက် ပြန်လည်ပြင်ဆင်ပါ။
1. စိတ်ကျေနပ်သောအခါ fine-tuned မော်ဒယ်ကို inference အတွက် ထုတ်လွှင့်ပါ။

မူလမော်ဒယ်အားလုံးသည် fine-tuning ကို ထောက်ပံ့မထားနိုင်ကြောင်း သတိပြုပါ - နောက်ဆုံးရသတင်းအချက်အလက်များအတွက် [OpenAI စာရွက်စာတမ်း](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) ကို စစ်ဆေးပါ။ ယခင်က fine-tuned ပြီးသော မော်ဒယ်ကိုလည်း ထပ်မံ fine-tune လုပ်နိုင်သည်။ ဤသင်ခန်းစာတွင် `gpt-35-turbo` ကို fine-tuning အတွက် ရည်ရွယ်ထားသော မူလမော်ဒယ်အဖြစ် အသုံးပြုမည်ဖြစ်သည်။

---


### အဆင့် 1.1: သင့်ဒေတာစုံကို ပြင်ဆင်ပါ

အရာဝတ္ထုတစ်ခုအကြောင်း မေးခွန်းများကို limerick ဖြင့် ဖြေဆိုပေးကာ သင့်အား အရာဝတ္ထုများ၏ ကာလဇယားကို နားလည်နိုင်စေမည့် chatbot တစ်ခု တည်ဆောက်ကြမယ်။ _ဒီ_ ရိုးရှင်းတဲ့ သင်ခန်းစာမှာတော့ ဒေတာစုံတစ်ခုကို ဖန်တီးပြီး မော်ဒယ်ကို လေ့ကျင့်ရန် အဖြေများ၏ မျှော်မှန်းထားသော ပုံစံကို ပြသသည့် နမူနာအချို့သာ ဖန်တီးပါမယ်။ အမှန်တကယ် အသုံးပြုမှုတွင်တော့ နမူနာများ ပိုများစွာပါဝင်သည့် ဒေတာစုံတစ်ခု ဖန်တီးရန် လိုအပ်ပါမယ်။ သင့်လျော်သော အက်ပလီကေးရှင်း ဒိုမိန်းအတွက် ရရှိနိုင်ပါက ဖွင့်လှစ်ထားသော ဒေတာစုံတစ်ခုကိုလည်း အသုံးပြုနိုင်ပြီး fine-tuning အတွက် ပြန်လည်ဖော်ပြနိုင်ပါသည်။

`gpt-35-turbo` ကို ဦးတည်ပြီး တစ်ကြိမ်တည်းဖြေဆိုမှု (chat completion) ရှာဖွေနေသောကြောင့် OpenAI chat completion လိုအပ်ချက်များကို ပြသသည့် [ဒီအကြံပြုထားသော ပုံစံ](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) ကို အသုံးပြု၍ နမူနာများ ဖန်တီးနိုင်ပါသည်။ မျိုးစုံပြောဆိုမှု (multi-turn conversational content) များကို မျှော်လင့်ပါက fine-tuning လုပ်ငန်းစဉ်တွင် အသုံးပြုသင့်/မသင့် မက်ဆေ့ခ်ျများကို အချက်ပြရန် `weight` ပါရာမီတာပါဝင်သည့် [multi-turn နမူနာပုံစံ](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) ကို အသုံးပြုရပါမည်။

ဒီသင်ခန်းစာအတွက်တော့ ရိုးရှင်းသော တစ်ကြိမ်တည်းဖြေဆိုမှု ပုံစံကို အသုံးပြုပါမယ်။ ဒေတာသည် တစ်ကြောင်းလျှင် ၁ မှတ်တမ်းရှိသည့် JSON ပုံစံအရာဝတ္ထုဖြစ်သည့် [jsonl ပုံစံ](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) ဖြင့် ရေးသားထားသည်။ အောက်တွင် နမူနာအဖြစ် ၂ မှတ်တမ်းကို ပြထားပြီး ကျွန်ုပ်တို့ fine-tuning သင်ခန်းစာအတွက် အသုံးပြုမည့် နမူနာအပြည့်အစုံ (၁၀ နမူနာ) ကို [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) တွင် ကြည့်ရှုနိုင်ပါသည်။ **မှတ်ချက်။** တစ်ခုချင်းစီမှတ်တမ်းကို တစ်ကြောင်းတည်းတွင် သတ်မှတ်ရမည် (ပုံမှန် JSON ဖိုင်ကဲ့သို့ အကြောင်းအရာများကို မခွဲခြားရပါ)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

အမှန်တကယ် အသုံးပြုမှုတွင် ကောင်းမွန်သောရလဒ်ရရှိရန် နမူနာများ ပိုများစွာ လိုအပ်ပါမည် - အဖြေများ၏ အရည်အသွေးနှင့် fine-tuning အချိန်/ကုန်ကျစရိတ်တို့အကြား သဘောတူညီမှုရှိရမည်။ ကျွန်ုပ်တို့သည် လုပ်ငန်းစဉ်ကို ရှင်းလင်းပြသရန် အမြန်ဆုံး fine-tuning ပြုလုပ်နိုင်ရန် နမူနာအနည်းငယ်သာ အသုံးပြုနေပါသည်။ ပိုမိုရှုပ်ထွေးသော fine-tuning သင်ခန်းစာအတွက် [ဒီ OpenAI Cookbook နမူနာ](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) ကို ကြည့်ရှုနိုင်ပါသည်။


---

### အဆင့် 1.2 သင့်ဒေတာစနစ်ကိုတင်ပါ

ဒေတာကို Files API ကို အသုံးပြု၍ [ဒီမှာဖော်ပြထားသလို](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) တင်ပါ။ ဤကုဒ်ကို လည်ပတ်ရန်အတွက် အောက်ပါအဆင့်များကို အရင်ဆုံး ပြုလုပ်ထားရမည်ဖြစ်သည်။
 - `openai` Python package ကို ထည့်သွင်းထားရမည် (နောက်ဆုံးအင်္ဂါရပ်များအတွက် ဗားရှင်း >=0.28.0 ကို သုံးပါ)
 - သင့် OpenAI API key ကို `OPENAI_API_KEY` ပတ်ဝန်းကျင်အပြောင်းအလဲအဖြစ် သတ်မှတ်ထားရမည်
ပိုမိုသိရှိလိုပါက သင်တန်းအတွက် ပေးထားသော [Setup guide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ကို ကြည့်ပါ။

ယခု၊ သင့်ဒေသခံ JSONL ဖိုင်မှ တင်ရန်ဖိုင်တစ်ခု ဖန်တီးရန် ကုဒ်ကို လည်ပတ်ပါ။


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### အဆင့် 2.1: SDK ဖြင့် Fine-tuning အလုပ်ကို ဖန်တီးပါ


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### အဆင့် 2.2: အလုပ်အခြေအနေကို စစ်ဆေးခြင်း

`client.fine_tuning.jobs` API ဖြင့် ပြုလုပ်နိုင်သည့် အရာများမှာ အောက်ပါအတိုင်းဖြစ်သည်-
- `client.fine_tuning.jobs.list(limit=<n>)` - နောက်ဆုံး fine-tuning အလုပ် n ခုကို စာရင်းပြုစုသည်
- `client.fine_tuning.jobs.retrieve(<job_id>)` - အထူး fine-tuning အလုပ်တစ်ခု၏ အသေးစိတ်ကို ရယူသည်
- `client.fine_tuning.jobs.cancel(<job_id>)` - fine-tuning အလုပ်တစ်ခုကို ပယ်ဖျက်သည်
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - အလုပ်မှ ဖြစ်ရပ်များ n ခုအထိ စာရင်းပြုစုသည်
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

လုပ်ငန်းစဉ်၏ ပထမအဆင့်မှာ _သင်ကြားရေးဖိုင်ကို မှန်ကန်မှုစစ်ဆေးခြင်း_ ဖြစ်ပြီး ဒေတာသည် မှန်ကန်သော ပုံစံဖြစ်ကြောင်း သေချာစေရန်ဖြစ်သည်။


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### အဆင့် 2.3: တိုးတက်မှုကို စောင့်ကြည့်ရန် ဖြစ်ရပ်များကို မှတ်တမ်းတင်ပါ


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### အဆင့် ၂.၄: OpenAI Dashboard တွင် အခြေအနေကို ကြည့်ရှုပါ


သင်သည် OpenAI ဝက်ဘ်ဆိုက်သို့ သွားရောက်ပြီး ပလက်ဖောင်း၏ _Fine-tuning_ အပိုင်းကို စူးစမ်းကြည့်ရှုခြင်းဖြင့်လည်း အခြေအနေကို ကြည့်ရှုနိုင်ပါသည်။ ၎င်းသည် လက်ရှိအလုပ်၏ အခြေအနေကို ပြသပေးမည်ဖြစ်ပြီး၊ ယခင်အလုပ်ဆောင်ရွက်မှုများ၏ သမိုင်းကြောင်းကိုလည်း လိုက်လံကြည့်ရှုနိုင်စေပါသည်။ ဤ screenshot တွင် ယခင်အလုပ်ဆောင်ရွက်မှု မအောင်မြင်ခဲ့ကြောင်းနှင့် ဒုတိယအလုပ်ဆောင်ရွက်မှု အောင်မြင်ခဲ့ကြောင်းကို တွေ့နိုင်ပါသည်။ အကြောင်းအရာအနေဖြင့်၊ ပထမဆုံးအလုပ်ဆောင်ရွက်မှုသည် မှားယွင်းစွာ ဖော်ပြထားသော JSON ဖိုင်ကို အသုံးပြုခဲ့သောအခါ ဖြစ်ပွားခဲ့ပြီး - ပြင်ဆင်ပြီးနောက် ဒုတိယအလုပ်ဆောင်ရွက်မှု အောင်မြင်စွာ ပြီးစီးခဲ့ပြီး မော်ဒယ်ကို အသုံးပြုနိုင်ရန် ရရှိစေခဲ့ပါသည်။

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba.my.png)


သင်သည် အောက်သို့ ဆွဲချလိုက်ခြင်းဖြင့် ဗီဇွယ် ဒက်ရှ်ဘုတ်တွင် အခြေအနေစာတိုက်များနှင့် မက်ထရစ်များကိုလည်း ကြည့်ရှုနိုင်ပါသည်။

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b.my.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a65229.my.png)|


---

### အဆင့် 3.1: ID ကို ရယူပြီး Fine-Tuned မော်ဒယ်ကို ကုဒ်ထဲတွင် စမ်းသပ်ခြင်း


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### အဆင့် 3.2: Playground တွင် Fine-Tuned မော်ဒယ်ကို လုပ်ဆောင်ပြီး စမ်းသပ်ခြင်း

ယခုအခါ သင်သည် fine-tuned မော်ဒယ်ကို နည်းလမ်းနှစ်မျိုးဖြင့် စမ်းသပ်နိုင်ပါပြီ။ ပထမတစ်ခုမှာ Playground သို့ သွားပြီး Models drop-down မှ သင့်အသစ် fine-tuned လုပ်ထားသော မော်ဒယ်ကို ရွေးချယ်နိုင်သည်။ နောက်တစ်ခုမှာ Fine-tuning panel တွင် ပြသထားသော "Playground" ရွေးချယ်မှုကို အသုံးပြုခြင်းဖြစ်ပြီး (အထက်ပါ screenshot ကို ကြည့်ပါ) အောက်ပါ _နှိုင်းယှဉ်မှု_ မြင်ကွင်းကို စတင်ပေးပြီး foundation နှင့် fine-tuned မော်ဒယ်ဗားရှင်းများကို ဘေးဘေးတွင် မြန်ဆန်စွာ သုံးသပ်နိုင်သည်။

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016.my.png)

သင်၏ လေ့ကျင့်မှုဒေတာတွင် အသုံးပြုထားသော system context ကို ဖြည့်စွက်ပြီး စမ်းသပ်မေးခွန်းကို ပေးပါ။ နှစ်ဖက်စလုံးတွင် တူညီသော context နှင့် မေးခွန်းများ ပြောင်းလဲသွားသည်ကို တွေ့မြင်ရမည်။ နှိုင်းယှဉ်မှုကို လုပ်ဆောင်ပြီး ထွက်ရှိလာသော အဖြေများအကြား ကွာခြားချက်ကို ကြည့်ရှုနိုင်ပါသည်။ _fine-tuned မော်ဒယ်သည် သင်ပေးထားသော ဥပမာများအတိုင်း အဖြေကို ဖော်ပြပေးသည့်အခါ foundation မော်ဒယ်သည် system prompt ကိုသာ လိုက်နာသွားသည်ကို သတိပြုပါ_။

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350.my.png)

နှိုင်းယှဉ်မှုတွင် မော်ဒယ်တိုင်းအတွက် token အရေအတွက်နှင့် inference အချိန်ကိုလည်း ပြသပေးသည်ကို တွေ့မြင်ရမည်။ **ဤနမူနာသည် လုပ်ငန်းစဉ်ကို ပြသရန် ရိုးရှင်းသော နမူနာတစ်ခုဖြစ်ပြီး အမှန်တကယ် ကမ္ဘာ့ဒေတာသို့မဟုတ် အခြေအနေတစ်ခုကို မဖော်ပြပါ**။ နှစ်ဖက်စလုံးတွင် token အရေအတွက်တူညီသည်ကို တွေ့ရမည် (system context နှင့် user prompt တူညီသည်) fine-tuned မော်ဒယ်သည် inference အတွက် ပိုမိုအချိန်ယူသည် (custom မော်ဒယ်)။

အမှန်တကယ် ကမ္ဘာ့အခြေအနေများတွင် ဤကဲ့သို့သော ကစားစရာနမူနာကို မသုံးဘဲ အမှန်တကယ် ဒေတာများ (ဥပမာ - ဖောက်သည်ဝန်ဆောင်မှုအတွက် ထုတ်ကုန်စာရင်း) ဖြင့် fine-tuning လုပ်မည်ဖြစ်ပြီး အဖြေ၏ အရည်အသွေး ပိုမိုထင်ရှားလာမည်ဖြစ်သည်။ _အဲဒီအခြေအနေတွင် foundation မော်ဒယ်ဖြင့် တူညီသော အရည်အသွေးရရှိရန် custom prompt engineering ပိုမိုလိုအပ်ပြီး token အသုံးပြုမှုနှင့် inference အချိန် ပိုမိုတိုးပွားနိုင်သည်_။ _ဤကို စမ်းသပ်ရန် OpenAI Cookbook တွင် fine-tuning နမူနာများကို ကြည့်ရှုနိုင်ပါသည်_။

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**အကြောင်းကြားချက်**  
ဤစာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ဖြင့် ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မှားယွင်းချက်များ ပါဝင်နိုင်ကြောင်း သတိပြုပါရန် မေတ္တာရပ်ခံအပ်ပါသည်။ မူရင်းစာတမ်းကို မိမိဘာသာစကားဖြင့်သာ တရားဝင်အရင်းအမြစ်အဖြစ် ယူဆသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်မှ ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကို အသုံးပြုရာမှ ဖြစ်ပေါ်လာနိုင်သည့် နားလည်မှုမှားယွင်းမှုများ သို့မဟုတ် မှားဖတ်ရှုမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
